In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

In [2]:
# Iniciar el navegador (asegúrate de tener un controlador como ChromeDriver instalado)

options = webdriver.ChromeOptions()

options.add_argument("--headless")  # Ejecuta el navegador en modo headless


driver = webdriver.Chrome(options=options)

# Navegar a la página
driver.get("https://www.baseball-reference.com/leagues/majors/2024-schedule.shtml")

# Esperar a que el elemento esté presente
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "section_wrapper")))

# Obtener el HTML de la página
page_content = driver.page_source

# Parsear el HTML con BeautifulSoup
soup = BeautifulSoup(page_content, "html.parser")

# Buscar la tabla de bateo
table_content = soup.find("div",{"class":"section_content"})

#Buscar los boxscores

boxscores = table_content.find_all("em")


driver.quit()

In [3]:
# Extraer los href de los enlaces dentro de <em>
hrefs = [a.find("a")["href"] for a in boxscores if a.find("a")]

In [4]:
def match_scrapping(soup):

    #Get visiting and local teams
    vteam = soup.find("div",{"class":"scorebox"}).find_all("strong")[0].text.strip()
    lteam = soup.find("div",{"class":"scorebox"}).find_all("strong")[1].text.strip()
    
    vteam_id= vteam.replace(' ', '').replace('.', '')
    lteam_id= lteam.replace(' ', '').replace('.', '')

    #Get the metadata of the game
    date_content = soup.find("div",{"class":"scorebox_meta"}).find_all("div")

    date=date_content[0].text
    time=date_content[1].text
    attendance=date_content[2].text
    venue=date_content[3].text

    #Get the data for the visiting team
    vteam_content = soup.find(id=f"{vteam_id}batting")
    vteam_content_tr = vteam_content.find_all("tr")

    l = []
    for tr in vteam_content_tr:
        td = tr.find_all('td')
        th = tr.find_all('th')
        row = [tr.text for tr in td]
        team  = [tr.text for tr in th]
        complete_row = team + row
        l.append(complete_row)

    batting_vteam=pd.DataFrame(l[1:-1], columns=l[0])

    batting_vteam["Date"]=date
    batting_vteam["Time"]=time
    batting_vteam["Attendace"]=attendance
    batting_vteam["Venue"]=venue
    batting_vteam["Team"]=vteam

    #Get the data for the local team
    lteam_content = soup.find(id=f"{lteam_id}batting")
    lteam_content_tr = lteam_content.find_all("tr")

    l=[]
    for tr in lteam_content_tr:
        td = tr.find_all('td')
        th = tr.find_all('th')
        row = [tr.text for tr in td]
        team  = [tr.text for tr in th]
        complete_row = team + row
        l.append(complete_row)

    batting_lteam=pd.DataFrame(l[1:-1], columns=l[0])

    batting_lteam["Date"]=date
    batting_lteam["Time"]=time
    batting_lteam["Attendace"]=attendance
    batting_lteam["Venue"]=venue
    batting_lteam["Team"]=lteam

    batting_match_df=pd.concat([batting_vteam,batting_lteam], ignore_index=True)

    # Convert non-numeric values in the 'AB' column to NaN
    batting_match_df["AB"] = pd.to_numeric(batting_match_df["AB"], errors='coerce')

    # Drop rows with NaN in the 'AB' column
    batting_match_df = batting_match_df.dropna(subset=["AB"])


    #Get the ´pitching data for the visiting team
    vteam_content = soup.find(id=f"{vteam_id}pitching")
    vteam_content_tr = vteam_content.find_all("tr")

    l = []
    for tr in vteam_content_tr:
        td = tr.find_all('td')
        th = tr.find_all('th')
        row = [tr.text for tr in td]
        team  = [tr.text for tr in th]
        complete_row = team + row
        l.append(complete_row)

    pitching_vteam=pd.DataFrame(l[1:-1], columns=l[0])

    pitching_vteam["Date"]=date
    pitching_vteam["Time"]=time
    pitching_vteam["Attendace"]=attendance
    pitching_vteam["Venue"]=venue
    pitching_vteam["Team"]=vteam

    #Get the data for the local team
    lteam_content = soup.find(id=f"{lteam_id}pitching")
    lteam_content_tr = lteam_content.find_all("tr")

    l=[]
    for tr in lteam_content_tr:
        td = tr.find_all('td')
        th = tr.find_all('th')
        row = [tr.text for tr in td]
        team  = [tr.text for tr in th]
        complete_row = team + row
        l.append(complete_row)

    pitching_lteam=pd.DataFrame(l[1:-1], columns=l[0])

    pitching_lteam["Date"]=date
    pitching_lteam["Time"]=time
    pitching_lteam["Attendace"]=attendance
    pitching_lteam["Venue"]=venue
    pitching_lteam["Team"]=lteam


    pitching_match_df=pd.concat([pitching_vteam,pitching_lteam], ignore_index=True)
    
    return batting_match_df , pitching_match_df

In [6]:
pitching_stats_list=[]
batting_stats_list=[]

i=1

total_match=len(hrefs)

# Iniciar el navegador (asegúrate de tener un controlador como ChromeDriver instalado)
options = webdriver.ChromeOptions()

options.add_argument("--headless")  # Ejecuta el navegador en modo headless


driver = webdriver.Chrome(options=options)

try:

    for match in hrefs:
        # Navegar a la página

        print(f"Procesando:{match}")
        print(f"Partido número: {i}")
        
        porcentaje_restante = i/total_match*100
        print(f"Progreso: {round(porcentaje_restante,4)}% \n")

        driver.get(f"https://www.baseball-reference.com{match}")

        # Esperar a que el elemento esté presente
        WebDriverWait(driver, 10)#.until(EC.presence_of_element_located((By.CLASS_NAME, "section_wrapper")))

        # Obtener el HTML de la página
        page_content = driver.page_source

        # Parsear el HTML con BeautifulSoup
        soup = BeautifulSoup(page_content, "html.parser")

        df_batting , df_pitching=match_scrapping(soup)

        pitching_stats_list.append(df_pitching)
        batting_stats_list.append(df_batting)
        
        i=i+1

        # Pausa entre solicitudes para evitar sobrecargar el servidor
        time.sleep(5)
finally:  
    driver.quit()

# Concatenar todos los DataFrames acumulados
batting_stats = pd.concat(batting_stats_list, ignore_index=True)
pitching_stats = pd.concat(pitching_stats_list, ignore_index=True)

Procesando:/boxes/SDN/SDN202403200.shtml
Partido número: 1
Progreso: 0.0412% 

Procesando:/boxes/LAN/LAN202403210.shtml
Partido número: 2
Progreso: 0.0823% 

Procesando:/boxes/ARI/ARI202403280.shtml
Partido número: 3
Progreso: 0.1235% 

Procesando:/boxes/BAL/BAL202403280.shtml
Partido número: 4
Progreso: 0.1647% 

Procesando:/boxes/CHA/CHA202403280.shtml
Partido número: 5
Progreso: 0.2058% 

Procesando:/boxes/CIN/CIN202403280.shtml
Partido número: 6
Progreso: 0.247% 

Procesando:/boxes/HOU/HOU202403280.shtml
Partido número: 7
Progreso: 0.2882% 

Procesando:/boxes/KCA/KCA202403280.shtml
Partido número: 8
Progreso: 0.3294% 

Procesando:/boxes/LAN/LAN202403280.shtml
Partido número: 9
Progreso: 0.3705% 

Procesando:/boxes/MIA/MIA202403280.shtml
Partido número: 10
Progreso: 0.4117% 

Procesando:/boxes/OAK/OAK202403280.shtml
Partido número: 11
Progreso: 0.4529% 

Procesando:/boxes/SDN/SDN202403280.shtml
Partido número: 12
Progreso: 0.494% 

Procesando:/boxes/SEA/SEA202403280.shtml
Partido nú

Procesando:/boxes/CHN/CHN202404050.shtml
Partido número: 104
Progreso: 4.2816% 

Procesando:/boxes/CIN/CIN202404050.shtml
Partido número: 105
Progreso: 4.3228% 

Procesando:/boxes/COL/COL202404050.shtml
Partido número: 106
Progreso: 4.3639% 

Procesando:/boxes/DET/DET202404050.shtml
Partido número: 107
Progreso: 4.4051% 

Procesando:/boxes/KCA/KCA202404050.shtml
Partido número: 108
Progreso: 4.4463% 

Procesando:/boxes/MIL/MIL202404050.shtml
Partido número: 109
Progreso: 4.4874% 

Procesando:/boxes/NYA/NYA202404050.shtml
Partido número: 110
Progreso: 4.5286% 

Procesando:/boxes/PIT/PIT202404050.shtml
Partido número: 111
Progreso: 4.5698% 

Procesando:/boxes/SFN/SFN202404050.shtml
Partido número: 112
Progreso: 4.611% 

Procesando:/boxes/TEX/TEX202404050.shtml
Partido número: 113
Progreso: 4.6521% 

Procesando:/boxes/WAS/WAS202404050.shtml
Partido número: 114
Progreso: 4.6933% 

Procesando:/boxes/ANA/ANA202404060.shtml
Partido número: 115
Progreso: 4.7345% 

Procesando:/boxes/ATL/ATL2024

Procesando:/boxes/BOS/BOS202404130.shtml
Partido número: 206
Progreso: 8.4809% 

Procesando:/boxes/CHA/CHA202404130.shtml
Partido número: 207
Progreso: 8.522% 

Procesando:/boxes/CLE/CLE202404131.shtml
Partido número: 208
Progreso: 8.5632% 

Procesando:/boxes/CLE/CLE202404132.shtml
Partido número: 209
Progreso: 8.6044% 

Procesando:/boxes/DET/DET202404131.shtml
Partido número: 210
Progreso: 8.6455% 

Procesando:/boxes/DET/DET202404132.shtml
Partido número: 211
Progreso: 8.6867% 

Procesando:/boxes/HOU/HOU202404130.shtml
Partido número: 212
Progreso: 8.7279% 

Procesando:/boxes/LAN/LAN202404130.shtml
Partido número: 213
Progreso: 8.769% 

Procesando:/boxes/MIA/MIA202404130.shtml
Partido número: 214
Progreso: 8.8102% 

Procesando:/boxes/NYN/NYN202404130.shtml
Partido número: 215
Progreso: 8.8514% 

Procesando:/boxes/OAK/OAK202404130.shtml
Partido número: 216
Progreso: 8.8925% 

Procesando:/boxes/PHI/PHI202404130.shtml
Partido número: 217
Progreso: 8.9337% 

Procesando:/boxes/SEA/SEA20240

Procesando:/boxes/MIN/MIN202404200.shtml
Partido número: 307
Progreso: 12.6389% 

Procesando:/boxes/NYA/NYA202404200.shtml
Partido número: 308
Progreso: 12.6801% 

Procesando:/boxes/PHI/PHI202404200.shtml
Partido número: 309
Progreso: 12.7213% 

Procesando:/boxes/PIT/PIT202404200.shtml
Partido número: 310
Progreso: 12.7625% 

Procesando:/boxes/SDN/SDN202404200.shtml
Partido número: 311
Progreso: 12.8036% 

Procesando:/boxes/SFN/SFN202404200.shtml
Partido número: 312
Progreso: 12.8448% 

Procesando:/boxes/SLN/SLN202404200.shtml
Partido número: 313
Progreso: 12.886% 

Procesando:/boxes/WAS/WAS202404200.shtml
Partido número: 314
Progreso: 12.9271% 

Procesando:/boxes/ATL/ATL202404210.shtml
Partido número: 315
Progreso: 12.9683% 

Procesando:/boxes/CHN/CHN202404210.shtml
Partido número: 316
Progreso: 13.0095% 

Procesando:/boxes/CIN/CIN202404210.shtml
Partido número: 317
Progreso: 13.0506% 

Procesando:/boxes/CLE/CLE202404210.shtml
Partido número: 318
Progreso: 13.0918% 

Procesando:/boxes

Procesando:/boxes/SFN/SFN202404270.shtml
Partido número: 408
Progreso: 16.797% 

Procesando:/boxes/TEX/TEX202404270.shtml
Partido número: 409
Progreso: 16.8382% 

Procesando:/boxes/TOR/TOR202404270.shtml
Partido número: 410
Progreso: 16.8794% 

Procesando:/boxes/ANA/ANA202404280.shtml
Partido número: 411
Progreso: 16.9205% 

Procesando:/boxes/ATL/ATL202404280.shtml
Partido número: 412
Progreso: 16.9617% 

Procesando:/boxes/BAL/BAL202404280.shtml
Partido número: 413
Progreso: 17.0029% 

Procesando:/boxes/BOS/BOS202404280.shtml
Partido número: 414
Progreso: 17.0441% 

Procesando:/boxes/CHA/CHA202404280.shtml
Partido número: 415
Progreso: 17.0852% 

Procesando:/boxes/COL/COL202404280.shtml
Partido número: 416
Progreso: 17.1264% 

Procesando:/boxes/DET/DET202404280.shtml
Partido número: 417
Progreso: 17.1676% 

Procesando:/boxes/MIA/MIA202404280.shtml
Partido número: 418
Progreso: 17.2087% 

Procesando:/boxes/MIL/MIL202404280.shtml
Partido número: 419
Progreso: 17.2499% 

Procesando:/boxes

...

Procesando:/boxes/ARI/ARI202408290.shtml
Partido número: 2003
Progreso: 82.4619% 

Procesando:/boxes/BOS/BOS202408290.shtml
Partido número: 2004
Progreso: 82.5031% 

Procesando:/boxes/CHA/CHA202408290.shtml
Partido número: 2005
Progreso: 82.5443% 

Procesando:/boxes/CIN/CIN202408290.shtml
Partido número: 2006
Progreso: 82.5854% 

Procesando:/boxes/COL/COL202408290.shtml
Partido número: 2007
Progreso: 82.6266% 

Procesando:/boxes/DET/DET202408290.shtml
Partido número: 2008
Progreso: 82.6678% 

Procesando:/boxes/HOU/HOU202408290.shtml
Partido número: 2009
Progreso: 82.7089% 

Procesando:/boxes/LAN/LAN202408290.shtml
Partido número: 2010
Progreso: 82.7501% 

Procesando:/boxes/MIL/MIL202408290.shtml
Partido número: 2011
Progreso: 82.7913% 

Procesando:/boxes/PHI/PHI202408290.shtml
Partido número: 2012
Progreso: 82.8324% 

Procesando:/boxes/SLN/SLN202408290.shtml
Partido número: 2013
Progreso: 82.8736% 

Procesando:/boxes/ANA/ANA202408300.shtml
Partido número: 2014
Progreso: 82.9148% 

Proc

Procesando:/boxes/OAK/OAK202409050.shtml
Partido número: 2102
Progreso: 86.5377% 

Procesando:/boxes/PIT/PIT202409050.shtml
Partido número: 2103
Progreso: 86.5788% 

Procesando:/boxes/SDN/SDN202409050.shtml
Partido número: 2104
Progreso: 86.62% 

Procesando:/boxes/SFN/SFN202409050.shtml
Partido número: 2105
Progreso: 86.6612% 

Procesando:/boxes/TBA/TBA202409050.shtml
Partido número: 2106
Progreso: 86.7023% 

Procesando:/boxes/TEX/TEX202409050.shtml
Partido número: 2107
Progreso: 86.7435% 

Procesando:/boxes/ATL/ATL202409060.shtml
Partido número: 2108
Progreso: 86.7847% 

Procesando:/boxes/BAL/BAL202409060.shtml
Partido número: 2109
Progreso: 86.8259% 

Procesando:/boxes/BOS/BOS202409060.shtml
Partido número: 2110
Progreso: 86.867% 

Procesando:/boxes/CHN/CHN202409060.shtml
Partido número: 2111
Progreso: 86.9082% 

Procesando:/boxes/HOU/HOU202409060.shtml
Partido número: 2112
Progreso: 86.9494% 

Procesando:/boxes/KCA/KCA202409060.shtml
Partido número: 2113
Progreso: 86.9905% 

Procesa

Procesando:/boxes/ARI/ARI202409130.shtml
Partido número: 2201
Progreso: 90.6134% 

Procesando:/boxes/ATL/ATL202409130.shtml
Partido número: 2202
Progreso: 90.6546% 

Procesando:/boxes/CHA/CHA202409130.shtml
Partido número: 2203
Progreso: 90.6958% 

Procesando:/boxes/CLE/CLE202409130.shtml
Partido número: 2204
Progreso: 90.7369% 

Procesando:/boxes/COL/COL202409130.shtml
Partido número: 2205
Progreso: 90.7781% 

Procesando:/boxes/DET/DET202409130.shtml
Partido número: 2206
Progreso: 90.8193% 

Procesando:/boxes/MIN/MIN202409130.shtml
Partido número: 2207
Progreso: 90.8604% 

Procesando:/boxes/NYA/NYA202409130.shtml
Partido número: 2208
Progreso: 90.9016% 

Procesando:/boxes/PHI/PHI202409130.shtml
Partido número: 2209
Progreso: 90.9428% 

Procesando:/boxes/PIT/PIT202409130.shtml
Partido número: 2210
Progreso: 90.9839% 

Procesando:/boxes/SEA/SEA202409130.shtml
Partido número: 2211
Progreso: 91.0251% 

Procesando:/boxes/SFN/SFN202409130.shtml
Partido número: 2212
Progreso: 91.0663% 

Proc

Procesando:/boxes/CIN/CIN202409200.shtml
Partido número: 2300
Progreso: 94.6892% 

Procesando:/boxes/HOU/HOU202409200.shtml
Partido número: 2301
Progreso: 94.7303% 

Procesando:/boxes/KCA/KCA202409200.shtml
Partido número: 2302
Progreso: 94.7715% 

Procesando:/boxes/LAN/LAN202409200.shtml
Partido número: 2303
Progreso: 94.8127% 

Procesando:/boxes/MIA/MIA202409200.shtml
Partido número: 2304
Progreso: 94.8538% 

Procesando:/boxes/MIL/MIL202409200.shtml
Partido número: 2305
Progreso: 94.895% 

Procesando:/boxes/NYN/NYN202409200.shtml
Partido número: 2306
Progreso: 94.9362% 

Procesando:/boxes/OAK/OAK202409200.shtml
Partido número: 2307
Progreso: 94.9774% 

Procesando:/boxes/SDN/SDN202409200.shtml
Partido número: 2308
Progreso: 95.0185% 

Procesando:/boxes/SLN/SLN202409200.shtml
Partido número: 2309
Progreso: 95.0597% 

Procesando:/boxes/TBA/TBA202409200.shtml
Partido número: 2310
Progreso: 95.1009% 

Procesando:/boxes/TEX/TEX202409200.shtml
Partido número: 2311
Progreso: 95.142% 

Proces

Procesando:/boxes/ANA/ANA202409280.shtml
Partido número: 2399
Progreso: 98.7649% 

Procesando:/boxes/ARI/ARI202409280.shtml
Partido número: 2400
Progreso: 98.8061% 

Procesando:/boxes/ATL/ATL202409280.shtml
Partido número: 2401
Progreso: 98.8473% 

Procesando:/boxes/BOS/BOS202409280.shtml
Partido número: 2402
Progreso: 98.8884% 

Procesando:/boxes/CHN/CHN202409280.shtml
Partido número: 2403
Progreso: 98.9296% 

Procesando:/boxes/CLE/CLE202409280.shtml
Partido número: 2404
Progreso: 98.9708% 

Procesando:/boxes/COL/COL202409280.shtml
Partido número: 2405
Progreso: 99.0119% 

Procesando:/boxes/DET/DET202409280.shtml
Partido número: 2406
Progreso: 99.0531% 

Procesando:/boxes/MIL/MIL202409280.shtml
Partido número: 2407
Progreso: 99.0943% 

Procesando:/boxes/MIN/MIN202409280.shtml
Partido número: 2408
Progreso: 99.1354% 

Procesando:/boxes/NYA/NYA202409280.shtml
Partido número: 2409
Progreso: 99.1766% 

Procesando:/boxes/SEA/SEA202409280.shtml
Partido número: 2410
Progreso: 99.2178% 

Proc

In [9]:
batting_stats.to_csv("batting_data.csv")
pitching_stats.to_csv("pitching_data.csv")